In [46]:
import wandb
import pickle
import time
from src.datahandle import DataLoader
from src.model_cnn import *
#hyper
split= 0.8
batch_size = 32
data_shape = 'original'

directory = '../../extras/data/A_training_given/training_data/'
training_folder = 'training_data'
training_labels_file= 'training_norm.csv'

In [2]:
training_object= DataLoader(
        directory,
        training_folder,
        training_labels_file
        )
#train test split
train,test = training_object.LoadModelData_info(
        split = split, 
        batch_size = batch_size)


In [3]:
X,Y = training_object.Load_batch(test[0], data_shape=data_shape)
X = jnp.array(X, dtype=jnp.float32)
Y = jnp.array(Y, dtype=jnp.float32)
params = pickle.load(open('../../pkls/submission2_cnn.pkl', 'rb'))


In [11]:
X[0:1].shape

(1, 240, 320, 3)

In [9]:
_, apply_fun = my_combinator(
    stax.Conv(5,(5,5), padding='SAME'),Relu_layer,
    stax.AvgPool((3,3)),

    stax.Conv(5,(5,5), padding='SAME'),Relu_layer,
    stax.AvgPool((3,3)),

    stax.Conv(5, (5,5),padding='SAME'),Relu_layer,
    stax.MaxPool((2,2)),

    my_Flatten(),
    my_Dense(2)
)
def loss_fn(params, x, y):
    predictions = apply_fun(params,x)
    return (1/len(x))*jnp.sum((predictions-y)**2)

In [18]:
loss = loss_fn(params,X,Y)

In [19]:
loss

DeviceArray(0.03919352, dtype=float32)

In [49]:
from PIL import Image
from numpy import asarray
def Load_batch_quiz(batch, data_shape=(-1,1)):
    X = []
    Image_order = []
    for instance in batch:
        link = instance[0]
        image = Image.open(link)
        data = asarray(image)
        if data.shape[2] == 4:
            data = data[:,:,:-1]
        if data_shape != 'original':
            data = data.reshape(data_shape)
        image_number = instance[1]
        X.append(data)
        Image_order.append(image_number)
    X = jnp.array(X)/255
    Image_order = np.array(Image_order)
    return X,Image_order

In [50]:
quiz_directory = '../../extras/data/C_testing_given/test_data/'
quiz_training_folder = 'test_data'
quiz_object= DataLoader(
        quiz_directory,
        quiz_training_folder,
        )
quiz_train = quiz_object.LoadQuizData_info()

In [58]:
X,image_order = Load_batch_quiz(quiz_train, data_shape=data_shape)

In [59]:
X[0]

DeviceArray([[[0.7254902 , 0.7019608 , 0.7372549 ],
              [0.6784314 , 0.654902  , 0.6901961 ],
              [0.6313726 , 0.62352943, 0.6627451 ],
              ...,
              [0.18039216, 0.19215687, 0.23137255],
              [0.16078432, 0.22352941, 0.21960784],
              [0.15686275, 0.21960784, 0.21568628]],

             [[0.7176471 , 0.70980394, 0.7137255 ],
              [0.60784316, 0.6       , 0.6039216 ],
              [0.6117647 , 0.61960787, 0.62352943],
              ...,
              [0.18039216, 0.1764706 , 0.2509804 ],
              [0.18039216, 0.19215687, 0.21568628],
              [0.18039216, 0.19215687, 0.21568628]],

             [[0.7176471 , 0.7137255 , 0.7254902 ],
              [0.5882353 , 0.5882353 , 0.59607846],
              [0.61960787, 0.6       , 0.60784316],
              ...,
              [0.1882353 , 0.1764706 , 0.2509804 ],
              [0.16470589, 0.20784314, 0.22745098],
              [0.16078432, 0.2       , 0.22352941]],

 

In [60]:
prds = np.array(apply_fun(params,X))

In [61]:
prds.shape,image_order.shape

((1020, 2), (1020,))

In [62]:
final_prd = np.column_stack((image_order,prds))

In [63]:
image_order,prds,final_prd

(array([348, 412, 374, ..., 345, 351, 437]),
 array([[0.6690372 , 1.002062  ],
        [0.49958917, 0.09506261],
        [0.56665945, 0.18968415],
        ...,
        [0.31509638, 0.5284257 ],
        [0.5840515 , 0.21010381],
        [0.37664872, 0.7758928 ]], dtype=float32),
 array([[3.48000000e+02, 6.69037223e-01, 1.00206196e+00],
        [4.12000000e+02, 4.99589175e-01, 9.50626060e-02],
        [3.74000000e+02, 5.66659451e-01, 1.89684153e-01],
        ...,
        [3.45000000e+02, 3.15096378e-01, 5.28425694e-01],
        [3.51000000e+02, 5.84051490e-01, 2.10103810e-01],
        [4.37000000e+02, 3.76648724e-01, 7.75892794e-01]]))

In [64]:
import pandas as pd
final_ordered = final_prd[final_prd[:, 0].argsort()]
df = pd.DataFrame(final_ordered, columns = ['image_id','angle','speed'])
df = df.astype({'image_id': 'int32'})
df

,image_id,angle,speed
0,1,0.550736,0.452761
1,2,0.747539,1.000822
2,3,0.387601,0.750408
3,4,0.402067,0.827767
4,5,0.378613,0.724797
...,...,...,...
1015,1016,0.473031,1.050147
1016,1017,0.761413,0.530285
1017,1018,0.292220,0.998064
1018,1019,0.331269,0.441406


In [65]:
df.to_csv('submission.csv', index=False,)